## Seattle AirBNB dataset

## Questions of interest: Given a reviewer, how will he/she qualify each of the houses?
- can we predict price?
- can we predict reviews for a house?
- pick a reviewer random and predict qualification for any house.
- how many reviews from that user we need? Plot by number of reviews.


In [238]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import datetime

%matplotlib inline

In [239]:
#load data
calendar = pd.read_csv('seattle_calendar.csv')
listings = pd.read_csv('seattle_listings.csv')
reviews = pd.read_csv('seattle_reviews.csv')

In [240]:
#overview of shape and composition calendar
print(calendar.shape)
print(calendar.dtypes)
print(calendar.head(10))

(1393570, 4)
listing_id     int64
date          object
available     object
price         object
dtype: object
   listing_id        date available   price
0      241032  2016-01-04         t  $85.00
1      241032  2016-01-05         t  $85.00
2      241032  2016-01-06         f     NaN
3      241032  2016-01-07         f     NaN
4      241032  2016-01-08         f     NaN
5      241032  2016-01-09         f     NaN
6      241032  2016-01-10         f     NaN
7      241032  2016-01-11         f     NaN
8      241032  2016-01-12         f     NaN
9      241032  2016-01-13         t  $85.00


In [241]:
calendar.date.nunique()

365

For every house, we have the information of price and availability day by day

Data quality issues calendar df:
    - convert date to datetype
    - convert available to boolean
    - convert price to int/float

In [242]:
#overview of shape and composition reviews
print(reviews.shape)

(84849, 6)


In [243]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   listing_id     84849 non-null  int64 
 1   id             84849 non-null  int64 
 2   date           84849 non-null  object
 3   reviewer_id    84849 non-null  int64 
 4   reviewer_name  84849 non-null  object
 5   comments       84831 non-null  object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


In [244]:
#return counts greater than 1
reviews['reviewer_id'].value_counts().loc[lambda x : x>5]

206203      67
15121499    32
5775807     19
2734499     19
29590276    18
            ..
423381       6
594768       6
19235033     6
13120075     6
63815        6
Name: reviewer_id, Length: 160, dtype: int64

In [245]:
reviews.comments[16]

'Despite our late booking request, Rachel & Jon were very responsive and helpful over email. It was a great place to stay - the location was ideal, the house was clean, well-furnished, the room was cozy, and the cat made good company. Overall, a lovely experience and I would definitely recommend the Farmhouse! '

For every house, we have the reviews recieved and who made them

Data quality issues listings df:
    - convert date to datetype

In [246]:
#overview of shape and composition listings
print(listings.shape)
print(listings.dtypes)

(3818, 92)
id                                    int64
listing_url                          object
scrape_id                             int64
last_scraped                         object
name                                 object
                                     ...   
cancellation_policy                  object
require_guest_profile_picture        object
require_guest_phone_verification     object
calculated_host_listings_count        int64
reviews_per_month                   float64
Length: 92, dtype: object


In [247]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   listing_url                       3818 non-null   object 
 2   scrape_id                         3818 non-null   int64  
 3   last_scraped                      3818 non-null   object 
 4   name                              3818 non-null   object 
 5   summary                           3641 non-null   object 
 6   space                             3249 non-null   object 
 7   description                       3818 non-null   object 
 8   experiences_offered               3818 non-null   object 
 9   neighborhood_overview             2786 non-null   object 
 10  notes                             2212 non-null   object 
 11  transit                           2884 non-null   object 
 12  thumbn

Can be divided in sections, host, house description, neighbourhood, reviews

Quality issues in listings df:
    - drop license column DONE
    - drop neighbourhood, work with neighbourhood_group_cleansed and neighborhood_cleansed columns DONE
    - drop square feet DONE
    - neighborhood_overview copy according to neighbourhood_cleansed
    - space create dummy variable or count words
    - notes create dummy variable or count words
    - transit create dummy variable
    - thumbnail_url create dummy variable DONE
    - medium_url create dummy variable DONE
    - xl_picture_url create dummy variable DONE
    - host_about create count words
    - host_acceptance_rate, host_response_rate and host_response_time fill with mode DONE
    - host_is_superhost convert to bool DONE
    - weekly price and monthly price to int. a lot of nans DONE
    - security_deposit, cleaning_fee fill with 0 DONE
    - first and last review to dtype DONE
    - review_scores how to deal with houses without review? fill with mean? DONE

It would be interesting to create a reviewer table, get average reviews for each customer. Very few information about reviewers. Only 8% booked more than once.

How did they qualify?

In [248]:
listings.street

0           Gilman Dr W, Seattle, WA 98119, United States
1       7th Avenue West, Seattle, WA 98119, United States
2       West Lee Street, Seattle, WA 98119, United States
3       8th Avenue West, Seattle, WA 98119, United States
4            14th Ave W, Seattle, WA 98119, United States
                              ...                        
3813    Northwest 48th Street, Seattle, WA 98107, Unit...
3814    Fuhrman Avenue East, Seattle, WA 98102, United...
3815    South Laurel Street, Seattle, WA 98178, United...
3816    43rd Avenue East, Seattle, WA 98112, United St...
3817    Westlake Avenue North, Seattle, WA 98109, Unit...
Name: street, Length: 3818, dtype: object

### Wrangle data

#### calendar df

In [249]:
#convert to boolean
d = {'t': True, 'f': False}
calendar['available'] = calendar["available"].replace(d)

In [250]:
#convert date to datetime
calendar['date'] = pd.to_datetime(calendar['date'])

In [251]:
#convert price to int
calendar['price'] = calendar.price.str.replace('$', '').str.replace(',', '').astype(float)

In [252]:
calendar.head()

,listing_id,date,available,price
0,241032,2016-01-04,True,85.0
1,241032,2016-01-05,True,85.0
2,241032,2016-01-06,False,NaN
3,241032,2016-01-07,False,NaN
4,241032,2016-01-08,False,NaN


#### reviews df

In [253]:
#reviews date to datetype
reviews['date'] = pd.to_datetime(reviews['date'])

#### listings df

In [254]:
#drop license, neighbourhood and square feet columns
listings = listings.drop(columns=['license', 'neighbourhood', 'square_feet', 'experiences_offered', 'market', 'country_code', 'country'])

In [255]:
#host_is_superhost convert to bool type
d = {'t': True, 'f': False}
listings['host_is_superhost'] = listings["host_is_superhost"].replace(d)
listings['has_availability'] = listings["has_availability"].replace(d)

In [256]:
#host_acceptance_rate, host_response_rate and host_response_time fill with mode
listings['host_acceptance_rate'] = listings['host_acceptance_rate'].fillna(listings['host_acceptance_rate'].mode()[0])
listings['host_response_rate'] = listings['host_response_rate'].fillna(listings['host_response_rate'].mode()[0])
listings['host_response_time'] = listings['host_response_time'].fillna(listings['host_response_time'].mode()[0])
listings['host_neighbourhood'] = listings['host_response_time'].fillna(listings['host_response_time'].mode()[0])

In [258]:
listings['host_acceptance_rate'] = listings.host_acceptance_rate.str.replace('%', '').astype(int)
listings['host_response_rate'] = listings.host_response_rate.str.replace('%', '').astype(int)
listings['security_deposit'] = listings.security_deposit.str.replace('$', '').str.replace(',','').fillna(0).astype(float)
listings['cleaning_fee'] = listings.cleaning_fee.str.replace('$', '').str.replace(',','').fillna(0).astype(float)
listings['extra_people'] = listings.extra_people.str.replace('$', '').str.replace(',','').fillna(0).astype(float)

AttributeError: Can only use .str accessor with string values!

In [ ]:
listings['security_deposit'] = listings.security_deposit.str.replace('%', '').astype(int)

In [202]:
#review scores fill with mean
listings['review_scores_rating'] = listings['review_scores_rating'].fillna(listings['review_scores_rating'].mean())
listings['review_scores_accuracy'] = listings['review_scores_accuracy'].fillna(listings['review_scores_accuracy'].mean())
listings['review_scores_cleanliness'] = listings['review_scores_cleanliness'].fillna(listings['review_scores_cleanliness'].mean())
listings['review_scores_checkin'] = listings['review_scores_checkin'].fillna(listings['review_scores_checkin'].mean())
listings['review_scores_communication'] = listings['review_scores_communication'].fillna(listings['review_scores_communication'].mean())
listings['review_scores_location'] = listings['review_scores_location'].fillna(listings['review_scores_location'].mean())
listings['review_scores_value'] = listings['review_scores_value'].fillna(listings['review_scores_value'].mean())
listings['reviews_per_month'] = listings['reviews_per_month'].fillna(listings['reviews_per_month'].mean())

In [203]:
#convert first and last review date to datetime
today = datetime.date.today()

listings['first_review'] = pd.to_datetime(listings['first_review']).fillna(today)
listings['last_review'] = pd.to_datetime(listings['last_review']).fillna(today)

In [204]:
#create dummy variable 1 it there is picture, 0 else
listings['thumbnail_url'] = listings.thumbnail_url.isna()
listings['medium_url'] = listings.thumbnail_url.isna()
listings['picture_url'] = listings.thumbnail_url.isna()
listings['xl_picture_url'] = listings.thumbnail_url.isna()

In [205]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 85 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   listing_url                       3818 non-null   object 
 2   scrape_id                         3818 non-null   int64  
 3   last_scraped                      3818 non-null   object 
 4   name                              3818 non-null   object 
 5   summary                           3641 non-null   object 
 6   space                             3249 non-null   object 
 7   description                       3818 non-null   object 
 8   neighborhood_overview             2786 non-null   object 
 9   notes                             2212 non-null   object 
 10  transit                           2884 non-null   object 
 11  thumbnail_url                     3818 non-null   bool   
 12  medium

In [206]:
listings.has_availability

0       True
1       True
2       True
3       True
4       True
        ... 
3813    True
3814    True
3815    True
3816    True
3817    True
Name: has_availability, Length: 3818, dtype: bool

In [207]:
#create dummy variables where we count text words in descriptions and return as column
listings['dummy_transit'] = listings.transit.str.len().fillna(0).astype(int)
listings['dummy_space'] = listings.space.str.len().fillna(0).astype(int)
listings['dummy_notes'] = listings.notes.str.len().fillna(0).astype(int)
listings['dummy_host_about'] = listings.host_about.str.len().fillna(0).astype(int)
listings['dummy_neighborhood_overview'] = listings.neighborhood_overview.str.len().fillna(0).astype(int)
listings['dummy_description'] = listings.description.str.len().fillna(0).astype(int)

In [ ]:
#replace 'weekly_price', 'monthly_price' with price where missing